[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vinculum3141-ship-it/bike-availability-data-science/blob/main/notebooks/example_data_exploration.ipynb)

# 🎓 Example: Data Loading and Exploration

**Purpose**: Demonstrate best practices for loading and exploring bike availability data  
**Module**: Module 02 - Data Acquisition  
**Author**: Course Template  
**Date**: 2024-01-15

---

## 📋 Overview

This notebook demonstrates:
- ✅ Proper setup and imports
- ✅ Loading data from CSV
- ✅ Basic data exploration
- ✅ Creating visualizations
- ✅ Following coding standards

## 🔧 Setup

In [ ]:
# ═══════════════════════════════════════════════════════════
# 1. IMPORTS
# ═══════════════════════════════════════════════════════════

# Standard library
import os
import sys
from datetime import datetime

# Third-party imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configure pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Set random seed
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

print("✅ Setup complete!")
print(f"📦 Pandas version: {pd.__version__}")
print(f"📦 NumPy version: {np.__version__}")

## ⚙️ Configuration

In [ ]:
# ═══════════════════════════════════════════════════════════
# 2. CONFIGURATION
# ═══════════════════════════════════════════════════════════

# Directory paths
DATA_DIR = 'data/raw'
OUTPUT_DIR = 'data/processed'

# Analysis parameters
FIGURE_SIZE = (14, 6)
DATE_RANGE_START = '2024-01-01'
DATE_RANGE_END = '2024-01-31'

# Station configuration
STATION_CAPACITY = 30  # Average station capacity

print("✅ Configuration set")
print(f"📁 Data directory: {DATA_DIR}")
print(f"📁 Output directory: {OUTPUT_DIR}")

## 📥 Generate Sample Data

For demonstration purposes, we'll create sample bike availability data. 

In a real project, you would load data from a file or API:
```python
df = pd.read_csv(os.path.join(DATA_DIR, 'bike_data.csv'))
```

In [ ]:
def generate_sample_bike_data(start_date, end_date, num_stations=5):
    """
    Generate sample bike availability data for demonstration.
    
    Args:
        start_date (str): Start date in 'YYYY-MM-DD' format
        end_date (str): End date in 'YYYY-MM-DD' format
        num_stations (int): Number of bike stations
        
    Returns:
        pd.DataFrame: Sample bike availability data
    """
    # Create hourly timestamps
    date_range = pd.date_range(start=start_date, end=end_date, freq='H')
    
    # Create station IDs
    stations = [f'STATION_{i:03d}' for i in range(1, num_stations + 1)]
    
    # Create data for each station
    data_list = []
    for station in stations:
        for timestamp in date_range:
            # Simulate daily pattern (more bikes in morning/evening)
            hour = timestamp.hour
            base_availability = STATION_CAPACITY / 2
            
            # Morning peak (7-9am): fewer bikes
            if 7 <= hour <= 9:
                mean_bikes = base_availability * 0.5
            # Evening peak (5-7pm): fewer bikes
            elif 17 <= hour <= 19:
                mean_bikes = base_availability * 0.6
            # Night (11pm-5am): more bikes
            elif hour >= 23 or hour <= 5:
                mean_bikes = base_availability * 1.2
            else:
                mean_bikes = base_availability
            
            # Add some randomness
            bikes_available = int(np.random.normal(mean_bikes, 3))
            bikes_available = np.clip(bikes_available, 0, STATION_CAPACITY)
            
            data_list.append({
                'timestamp': timestamp,
                'station_id': station,
                'bikes_available': bikes_available,
                'docks_available': STATION_CAPACITY - bikes_available,
                'capacity': STATION_CAPACITY
            })
    
    df = pd.DataFrame(data_list)
    return df


# Generate sample data
df = generate_sample_bike_data(DATE_RANGE_START, DATE_RANGE_END, num_stations=5)

print(f"✅ Generated {len(df):,} rows of sample data")
print(f"📅 Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")
print(f"🚉 Stations: {df['station_id'].nunique()}")

## 🔍 Data Inspection

In [ ]:
# Display first few rows
print("=" * 80)
print("📊 FIRST 10 ROWS")
print("=" * 80)
display(df.head(10))

In [ ]:
# Dataset information
print("\n" + "=" * 80)
print("📋 DATASET INFORMATION")
print("=" * 80)
df.info()

In [ ]:
# Summary statistics
print("\n" + "=" * 80)
print("📈 SUMMARY STATISTICS")
print("=" * 80)
display(df.describe())

In [ ]:
# Check for missing values
print("\n" + "=" * 80)
print("❓ MISSING VALUES")
print("=" * 80)

missing = df.isnull().sum()
missing_pct = (df.isnull().sum() / len(df) * 100).round(2)

missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Percentage': missing_pct
})

if missing.sum() > 0:
    display(missing_df[missing_df['Missing Count'] > 0])
else:
    print("✅ No missing values found!")

## 📊 Data Exploration

Let's explore patterns in the data.

### 1. Bike Availability Distribution

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=FIGURE_SIZE)

# Histogram
axes[0].hist(df['bikes_available'], bins=30, edgecolor='black', alpha=0.7)
axes[0].axvline(df['bikes_available'].mean(), color='red', linestyle='--', 
                label=f'Mean: {df["bikes_available"].mean():.1f}')
axes[0].set_xlabel('Bikes Available')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Bike Availability')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Box plot by station
df.boxplot(column='bikes_available', by='station_id', ax=axes[1])
axes[1].set_xlabel('Station ID')
axes[1].set_ylabel('Bikes Available')
axes[1].set_title('Bike Availability by Station')
plt.suptitle('')  # Remove automatic title

plt.tight_layout()
plt.show()

print(f"📊 Average bikes available: {df['bikes_available'].mean():.2f}")
print(f"📊 Median bikes available: {df['bikes_available'].median():.2f}")
print(f"📊 Std deviation: {df['bikes_available'].std():.2f}")

### 2. Temporal Patterns

In [ ]:
# Add temporal features for analysis
df['hour'] = df['timestamp'].dt.hour
df['day_of_week'] = df['timestamp'].dt.dayofweek
df['day_name'] = df['timestamp'].dt.day_name()

# Calculate hourly average across all stations
hourly_avg = df.groupby('hour')['bikes_available'].mean()

# Plot hourly pattern
fig, ax = plt.subplots(figsize=FIGURE_SIZE)
hourly_avg.plot(kind='bar', ax=ax, color='steelblue', edgecolor='black')
ax.set_xlabel('Hour of Day')
ax.set_ylabel('Average Bikes Available')
ax.set_title('Average Bike Availability by Hour of Day')
ax.grid(True, alpha=0.3, axis='y')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

print("📊 Peak hours (lowest availability):")
print(hourly_avg.nsmallest(3))
print("\n📊 Off-peak hours (highest availability):")
print(hourly_avg.nlargest(3))

### 3. Time Series Visualization

In [ ]:
# Plot time series for one station
station_to_plot = df['station_id'].iloc[0]
df_station = df[df['station_id'] == station_to_plot].copy()

fig, ax = plt.subplots(figsize=(16, 5))
ax.plot(df_station['timestamp'], df_station['bikes_available'], 
        linewidth=0.8, alpha=0.7)
ax.set_xlabel('Time')
ax.set_ylabel('Bikes Available')
ax.set_title(f'Bike Availability Over Time - {station_to_plot}')
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print(f"📈 Showing data for: {station_to_plot}")
print(f"📅 Time period: {df_station['timestamp'].min()} to {df_station['timestamp'].max()}")

### 4. Correlation Analysis

In [ ]:
# Create correlation matrix
correlation_features = ['bikes_available', 'docks_available', 'hour', 'day_of_week']
correlation_matrix = df[correlation_features].corr()

# Plot heatmap
fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, ax=ax, cbar_kws={'label': 'Correlation'})
ax.set_title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()

print("📊 Key correlations:")
print(correlation_matrix['bikes_available'].sort_values(ascending=False))

## 💾 Save Processed Data

Save the explored data with temporal features for future use.

In [ ]:
# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Save to CSV
output_file = os.path.join(OUTPUT_DIR, 'bike_data_with_features.csv')
df.to_csv(output_file, index=False)

print(f"✅ Saved processed data to: {output_file}")
print(f"📊 Saved {len(df):,} rows and {len(df.columns)} columns")
print(f"📋 Columns: {', '.join(df.columns.tolist())}")

---
## 📝 Summary

### Key Findings

1. **Distribution**: Bike availability follows a roughly normal distribution with mean of ~15 bikes per station
2. **Temporal Patterns**: 
   - Morning peak (7-9am): Lower availability due to commute demand
   - Evening peak (5-7pm): Lower availability due to return commute
   - Night hours: Higher availability as bikes are not in use
3. **Station Variation**: Some variability between stations, but overall consistent patterns
4. **Correlation**: Strong negative correlation between bikes_available and docks_available (expected)

### Data Quality

✅ No missing values  
✅ Consistent time series  
✅ Reasonable value ranges  
✅ All stations have complete data

### Next Steps

- [ ] Merge with weather data
- [ ] Engineer more features (lag features, rolling averages)
- [ ] Build baseline prediction model
- [ ] Explore seasonal patterns with more data

### Code Quality Notes

This notebook demonstrates:
- ✅ Clear section organization with headers
- ✅ Descriptive variable names
- ✅ Function docstrings
- ✅ Comments explaining complex logic
- ✅ Configuration constants at the top
- ✅ Proper visualization labels and titles
- ✅ Summary of findings

---
## 📚 References

- [Pandas Documentation](https://pandas.pydata.org/docs/)
- [Matplotlib Gallery](https://matplotlib.org/stable/gallery/index.html)
- [Seaborn Tutorial](https://seaborn.pydata.org/tutorial.html)
- [Course Coding Standards](../../docs/coding_standards.md)